In [1]:
# model orbits with python and jupyter widgets

In [26]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy as sp
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

def convert_r_v(r, v): # converts position and velocity vectors, r and v, into orbital elements
    unit_i = np.array([1, 0, 0])
    unit_j = np.array([0, 1, 0])
    unit_k = np.array([0, 0, 1])
    mag_r = np.linalg.norm(r)
    mag_v = np.linalg.norm(v)
    mu = 1
    h = np.cross(r, v)
    n = np.cross(unit_k, h)
    mag_n = np.linalg.norm(n)
    mag_h = np.linalg.norm(h)
    p = mag_h**2
    e = (1 / mu) * ((r * (mag_v**2 - (mu / mag_r))) - (v * np.dot(r, v)))
    mag_e = np.linalg.norm(e)
    i = np.arccos((np.dot(h, unit_k)) / mag_h)
    if mag_n == 0:
        little_omega = 'none'
        big_omega = 'none'
    else:
        little_omega = np.arccos(np.dot(n, e) / (mag_n * mag_e))
        big_omega = np.arccos(np.dot(n, unit_i) / mag_n)
    nu = np.arccos(np.dot(e, r) / (mag_e * mag_r))
    return (p, mag_e, i, little_omega, big_omega, nu)

def convert_orbital_perifocal(p, e, nu): # converts orbital elements into position and velocity vectors in the perifocal frame
    mu = 1
    mag_r = p / (1 + (e * np.cos(nu)))
    r = np.array([mag_r * np.cos(nu), mag_r * np.sin(nu), 0])
    v = np.array([((mu / p) ** (1 / 2)) * (-1 * np.sin(nu)), ((mu / p) ** (1 / 2)) * (e + np.cos(nu)), 0])
    return r, v

def convert_orbital_geocentric(p, e, i, little_omega, big_omega, nu): # converts orbital elements into perifocal then geocentric
    r, v = convert_orbital_perifocal(p, e, nu)
    if big_omega == 'none' or little_omega == 'none':
        return r, v
    else:
        trans_matrix = np.array([[(np.cos(big_omega) * np.cos(little_omega)) - (np.sin(big_omega) * np.sin(little_omega) * np.cos(i)),
                                  (-1 * np.cos(big_omega) * np.sin(little_omega)) - (np.sin(big_omega) * np.cos(little_omega) * np.cos(i)),
                                  np.sin(big_omega) * np.sin(i)],
                                 [(np.sin(big_omega) * np.cos(little_omega)) + (np.cos(big_omega) * np.sin(little_omega) * np.cos(i)),
                                  (-1 * np.sin(big_omega) * np.sin(little_omega)) + (np.cos(big_omega) * np.cos(little_omega) * np.cos(i)),
                                  -1 * np.cos(big_omega) * np.sin(i)],
                                 [np.sin(little_omega) * np.sin(i), np.cos(little_omega) * np.sin(i), np.cos(i)]])
        r = np.dot(r, np.transpose(trans_matrix))
        v = np.dot(v, np.transpose(trans_matrix))
    print ('r', r)
    print ('v', v)

In [28]:
interact(convert_orbital_geocentric, p = (0.01, 15), e = (0.0, 1.0), i = (0, 359), little_omega = (0, 359), big_omega = (0, 359), nu = (0, 359))

r [ 4.2245036  -2.52565496 -0.12266599]
v [ 0.24295682  0.49272251  0.04261599]
